It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1399 in data/validation/cats
- put the dogs pictures index 0-999 in data/train/dogs
- put the dog pictures index 1000-1399 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        
        cats/
            cat.0.jpg
            cat.1.jpg
            ...
        dogs/
            dog.0.jpg
            dog.1.jpg
            ...  
    validation/
        cats/
            cat.1000.jpg
            cat.1001.jpg
            ...
        dogs/
            dog.1000.jpg
            dog.1001.jpg
            ...

In [8]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

In [3]:
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'cats_and_dogs_small/train'
validation_data_dir = 'cats_and_dogs_small/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [10]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [6]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * 1000 + [1] * 1000)

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * 400 + [1] * 400)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [9]:
save_bottlebeck_features()
train_top_model()

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


C:\Users\boshi\AppData\Local\Temp/ipykernel_15764/3314447564.py:24: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = model.predict_generator(


Epoch 1/50
125/125 [==============================] - 5s 36ms/step - loss: 0.7286 - accuracy: 0.7620 - val_loss: 1.0811 - val_accuracy: 0.6650
Epoch 2/50
125/125 [==============================] - 4s 35ms/step - loss: 0.3677 - accuracy: 0.8560 - val_loss: 0.7396 - val_accuracy: 0.7800
Epoch 3/50
125/125 [==============================] - 1s 4ms/step - loss: 0.3066 - accuracy: 0.8750 - val_loss: 0.6071 - val_accuracy: 0.8037
Epoch 4/50
125/125 [==============================] - 4s 35ms/step - loss: 0.2676 - accuracy: 0.8930 - val_loss: 0.8095 - val_accuracy: 0.8175
Epoch 5/50
125/125 [==============================] - 1s 4ms/step - loss: 0.2493 - accuracy: 0.9040 - val_loss: 0.9538 - val_accuracy: 0.8012
Epoch 6/50
125/125 [==============================] - 4s 35ms/step - loss: 0.2318 - accuracy: 0.9075 - val_loss: 1.0203 - val_accuracy: 0.7975
Epoch 7/50
125/125 [==============================] - 1s 4ms/step - loss: 0.1874 - accuracy: 0.9310 - val_loss: 0.8641 - val_accuracy: 0.8188
Ep